In [1]:
import numpy as np
import pandas as pd

house_dataset = pd.read_csv('1976-2020-house.csv')

In [2]:
# This dataset contains only the general election house condidates.
# The candidates who has no names and were dropped only had a few votes. The winner is the candidate with the greatest
# number of votes so this should not be a problem. However, they code the candidatevotes so that if someone is running
# in an uncontested election, they only have one vote. Theoretically, even if there was a mistake in their data
# dropping candidates with no names should fix it rather than harm.
na_drop_dataset = house_dataset.dropna(subset=["candidate"]) 
clean_dataset = na_drop_dataset[na_drop_dataset["special"] == False]

In [3]:
# The dataset also contains fusion candidates, who run on multiple parties at the same time for an election
# and sum up their votes at the end. The only differences in these candidates is in their candidatevotes and party
# Here, we take candidates running in the same election but with different parties and sum their votes together
column = [x for x in clean_dataset.columns if (x != "candidatevotes" and x != "party")]
sum_fusion_candidates = clean_dataset.groupby(column, as_index=False, dropna=False)['candidatevotes'].sum()

In [4]:
# Here, for any given election, designated by a unique combination of year, state and district
# we sort the candidates so that the candidate with the most votes is first in the row
# Then we take the first candidate and assume that they've won.
# This process does not take into account exact ties.
fusion_winner = sum_fusion_candidates.groupby(["year", "state", "district"], sort=False)\
                .apply(lambda x: x.sort_values(['candidatevotes'], ascending=False)).reset_index(drop=True)
required_winner = fusion_winner.groupby(["year", "state", "district"], sort=False).first().reset_index(drop=False)

In [5]:
# To standardise the data, we lower all the cases of the letters.
# Also, in the dime dataset, they'll denote a district as 02, but in the house dataset, they'll denote it as 2
# Hence, that's what the zfill is for. The house dataset will also denote the sole district as 00, but the 
# dime dataset will denote it as 01
required_winner["candidate"] = required_winner["candidate"].astype(str).str.lower()
required_winner["district"] = required_winner["district"].apply(lambda x: str(x).zfill(2))
required_winner["district"] = required_winner["district"].replace({"00":"01"})

In [6]:
# This grabs the first and last names of the candidates in the house dataset, since some of the candidates will
# have nicknames or names in brackets. They might also have jr. as the last part of their name
# We also make sure that our names don't have any commas or full stops in them
required_winner['fixed_candidate'] = [x[0] + ' ' + x[-1] if x[-1] != 'jr.' else x[0] +' '\
                                      + x[-2] for x in required_winner['candidate'].str.split()]

required_winner['lname'] = [x[1] for x in required_winner['fixed_candidate'].str.split()]
required_winner['lname'] = required_winner['lname'].str.replace(',', '')
required_winner['lname'] = required_winner['lname'].str.replace('.', '')

required_winner['fname'] = [x[0] for x in required_winner['fixed_candidate'].str.split()]
required_winner['fname'] = required_winner['fname'].str.replace(',', '')
required_winner['fname'] = required_winner['fname'].str.replace('.', '')

<ipython-input-6-03b5b86e8295>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  required_winner['lname'] = required_winner['lname'].str.replace('.', '')
<ipython-input-6-03b5b86e8295>:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  required_winner['fname'] = required_winner['fname'].str.replace('.', '')


In [7]:
# This makes it so that the winners data only has years that the dime dataset has data on.
# This is the house dataset completed in terms of finding the winners.
required_winner = required_winner.query("year < 2017 and year > 1979")

In [8]:
import numpy
import pandas as pd

dime_dataset = pd.read_csv('dime_recipients_all_1979_2014.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1,5,6,12,13,14,15,16,17,18,19,22,23,37,55,56,57,58,59,60,63,65,67,68,69,70,71,74) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
# This is the dime recipient dataset for the house only.
house = dime_dataset[dime_dataset["seat"] == "federal:house"]

In [10]:
# Here, the problem with the dime dataset is that sometimes the district data will have three letter state names
# rather than two, so this standardises everything so that the districts and states match with the other house dataset
state_po = pd.DataFrame(house["district"].str[0:2])
state_po = state_po.rename(columns={'district': 'state_po'})
district = pd.DataFrame(house["district"].str[2:])
district_state = district.join(state_po)
house = house.drop(['district'], axis=1)
house_standard = district_state.merge(house, left_index=True, right_index=True)

In [11]:
# This standardises the last names and first names to also have no dots and commas as well.
house_standard['lname'] = house_standard['lname'].str.replace(',', '')
house_standard['lname'] = house_standard['lname'].str.replace('.', '')
house_standard['fname'] = house_standard['fname'].str.replace(',', '')
house_standard['fname'] = house_standard['fname'].str.replace('.', '')

<ipython-input-11-6b4ea01093a6>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  house_standard['lname'] = house_standard['lname'].str.replace('.', '')
<ipython-input-11-6b4ea01093a6>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  house_standard['fname'] = house_standard['fname'].str.replace('.', '')


In [12]:
# This grabs the important variables that we need to merge on and collect data for
important_columns = ['district', 'state_po','cycle', 'fecyear', 'ICPSR', 'ICPSR2', 'name', \
'lname', 'fname', 'party','Incum.Chall','recipient.cfscore', 'contributor.cfscore', 'recipient.cfscore.dyn',\
'dwnom1', 'dwnom2', 'ps.dwnom1', 'ps.dwnom2', 'dwdime', 'irt.cfscore','num.givers', 'num.givers.total',\
'n.data.points.personal.donations', 'n.data.points.personal.donations.unq', 'cand.gender', 'total.disbursements', \
'total.pc.contribs', 'contribs.from.candidate', 'unitemized', 'non.party.ind.exp.for', 'non.party.ind.exp.against',\
'ind.exp.for', 'ind.exp.against', 'comm.cost.for', 'comm.cost.against','party.coord.exp', 'party.ind.exp.against', \
'total.receipts', 'total.indiv.contrib', 'total.pac.contribs', 'ran.primary', 'ran.general', 'p.elec.stat', \
's.elec.stat', 'r.elec.stat','gen.elec.stat', 'gen.elect.pct', 'winner', 'district.partisanship', 'district.pres.vs',\
'candStatus',  'before.switch.ICPSR', 'after.switch.ICPSR', 'party.orig']

In [13]:
# here, we separate the dime data into two groups. The winners of house elections and the losers
# This has less than the house dataset
house_important = house_standard[important_columns]
house_winner = house_important[house_important["winner"] == "W"]
house_loser = house_important[house_important["winner"] != "W"]

In [14]:
# Since this house winner dataset contains only the winner, it seems logical that even if we merge by first name
# or last name only rather than full name, conditional on the same district, state and year
# then it would be really unlikely that there would be something miscoded here
filter_one = house_winner.merge(required_winner, how="right", left_on=['fname', 'district', 'state_po', 'cycle']\
                      , right_on = ['fname', 'district', 'state_po', 'year'])

In [15]:
# This merge only adds a single row, of a person whose name is steve stockman. He has can inconsistent ICPSR
# Since they have the same ICPSR2, I'm assuming perhaps that they switched names. I am note sure about this.
filter_one.iloc[7342:7344,:] # This is the duplicate candidate

,district,state_po,cycle,fecyear,ICPSR,ICPSR2,name,lname_x,fname,party,...,candidate,writein,mode,totalvotes,unofficial,version,fusion_ticket,candidatevotes,fixed_candidate,lname_y
7342,36,TX,2012.0,2012.0,H6TX221432012,29570,"stockman, steve",stockman,steve,200.0,...,steve stockman,False,TOTAL,233832,False,20210604,False,165405,steve stockman,stockman
7343,36,TX,2012.0,2012.0,295702012,29570,"stockman, stephen e",stockman,steve,200.0,...,steve stockman,False,TOTAL,233832,False,20210604,False,165405,steve stockman,stockman


In [16]:
# Here, we notice that the last names are inconsistent between the candidates. Since we care about the dime dataset data
# we choose to use lname_x as our primary last name
filter_one[(filter_one["lname_x"] != filter_one["lname_y"]) & (~filter_one["lname_x"].isna())]

,district,state_po,cycle,fecyear,ICPSR,ICPSR2,name,lname_x,fname,party,...,candidate,writein,mode,totalvotes,unofficial,version,fusion_ticket,candidatevotes,fixed_candidate,lname_y
14,03,AR,1980.0,1982.0,110201980,11020,"hammerschmidt, john paul",hammerschmidt,john,200,...,john paul hammerschimdt,False,TOTAL,1,False,20210604,False,1,john hammerschimdt,hammerschimdt
191,09,MI,1980.0,1980.0,107941980,10794,"vanderjagt, guy",vanderjagt,guy,200,...,guy vander jagt,False,TOTAL,174776,False,20210604,False,168713,guy jagt,jagt
213,04,MS,1980.0,1980.0,146341980,14634,"hinson, jon c.",hinson,jon,200,...,jon c. henson,False,TOTAL,177896,False,20210604,False,69321,jon henson,henson
335,02,PA,1980.0,1980.0,146291980,14629,"gray, william h iii",gray,william,100,...,william h. gray iii,False,TOTAL,131828,False,20210604,False,127106,william iii,iii
359,01,RI,1980.0,1980.0,105611980,10561,"st germain, fernand j",st germain,fernand,100,...,fernand j. st germain,False,TOTAL,178600,False,20210604,False,120756,fernand germain,germain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7202,03,NM,2012.0,2012.0,209322012,20932,"lujan, ben r",lujan,ben,100.0,...,ben bay luj_n,False,TOTAL,264719,False,20210604,False,167103,ben luj_n,luj_n
7209,07,NY,2012.0,2012.0,293782012,29378,"velazquez, nydia m",velazquez,nydia,100.0,...,nydia m. vela?zquez,False,TOTAL,178825,False,20210604,True,141322,nydia vela?zquez,vela?zquez
7362,03,WA,2012.0,2012.0,H0WA031872012,21187,"herrera beutler, jaime",herrera beutler,jaime,200.0,...,jaime herrera beutler,False,TOTAL,293884,False,20210604,False,177446,jaime beutler,beutler
7364,05,WA,2012.0,2012.0,205352012,20535,"mcmorris, cathy",mcmorris,cathy,200.0,...,cathy mcmorris rodgers,False,TOTAL,308578,False,20210604,False,191066,cathy rodgers,rodgers


In [17]:
# This makes it so that the last name is the one found in the dime dataset, except for the last names with null values
filter_one["lname"] = filter_one['lname_x'].where(filter_one['lname_x'].notnull(), filter_one['lname_y'])
# Here, we drop the useless columns and duplicate people
filter_one = filter_one.drop_duplicates(subset=['fname','district', 'state_po', 'year'], keep='first')
filter_one_dropped = filter_one.drop(['lname_x','lname_y'],axis=1)
filter_one_dropped_winner = filter_one_dropped[filter_one_dropped["winner"] == "W"]

In [18]:
filter_one_dropped_winner.to_excel("filter_one.xlsx") 

In [19]:
# This basically grabs only the losers and resets the columns so that a proper merge can be done again on the losers
# This time by last name.
filter_one_dropped_loser = filter_one_dropped[filter_one_dropped["winner"] != "W"]
filter_one_reset = filter_one_dropped_loser[required_winner.columns]

In [20]:
# In this merge, no duplicate columns were added. Only inconsistent first names
filter_two = house_winner.merge(filter_one_reset, how="right", left_on=['lname', 'district', 'state_po', 'cycle']\
                      , right_on = ['lname','district', 'state_po', 'year'])

In [21]:
# There are many inconsistent first names this time but that is to be expected
# We take the first names from the dime dataset, though it doesn't matter
filter_two[(filter_two['fname_x'] != filter_two['fname_y']) & (~filter_two["fname_x"].isna())]

,district,state_po,cycle,fecyear,ICPSR,ICPSR2,name,lname,fname_x,party,...,candidate,writein,mode,totalvotes,unofficial,version,fusion_ticket,candidatevotes,fixed_candidate,fname_y
0,02,AL,1980.0,1980.0,107171980,10717,"dickinson, hon. william l.",dickinson,hon,200,...,bill dickinson,False,TOTAL,172962,False,20210604,False,104796,bill dickinson,bill
4,02,CA,1980.0,1980.0,105791980,10579,"clausen, donald h.",clausen,donald,200,...,don h. clausen,False,TOTAL,261674,False,20210604,False,141698,don clausen,don
5,11,CA,1980.0,1982.0,148371980,14837,"lantos, thomas p",lantos,thomas,100,...,tom lantos,False,TOTAL,185012,False,20210604,False,85823,tom lantos,tom
7,15,CA,1980.0,1980.0,146131980,14613,"coelho, anthony lee",coelho,anthony,100,...,tony coelho,False,TOTAL,150491,False,20210604,False,108072,tony coelho,tony
10,34,CA,1980.0,1980.0,146471980,14647,"lungren, daniel e",lungren,daniel,200,...,dan lungren,False,TOTAL,192169,False,20210604,False,138024,dan lungren,dan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899,08,WA,2012.0,2012.0,205362012,20536,"reichert, dave",reichert,dave,200.0,...,david g. reichert,False,TOTAL,302090,False,20210604,False,180204,david reichert,david
1900,10,WA,2012.0,2012.0,H0WA031612012,21369,"heck, dennis",heck,dennis,100.0,...,denny heck,False,TOTAL,278417,False,20210604,False,163036,denny heck,denny
1901,03,WI,2012.0,2012.0,297692012,29769,"kind, ronald james",kind,ronald,100.0,...,ron kind,False,TOTAL,339764,False,20210604,False,217712,ron kind,ron
1902,04,WI,2012.0,2012.0,205372012,20537,"moore, gwendolynne",moore,gwendolynne,100.0,...,gwen moore,False,TOTAL,325788,False,20210604,False,235257,gwen moore,gwen


In [26]:
# This repeats the process of grabbing the winners only and then continuing on with people that were not winning
# or weren't matched
filter_two["fname"] = filter_two['fname_x'].where(filter_two['fname_x'].notnull(), filter_two['fname_y'])
filter_two_dropped = filter_two.drop(['fname_x','fname_y'],axis=1)
filter_two_dropped_winner = filter_two_dropped[filter_one_dropped["winner"] == "W"]

<ipython-input-26-49f3215096a5>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filter_two_dropped_winner = filter_two_dropped[filter_one_dropped["winner"] == "W"]


In [30]:
filter_two_dropped_winner.to_excel("filter_two.xlsx") 

In [31]:
filter_two_dropped_loser = filter_two_dropped[filter_one_dropped["winner"] != "W"]

<ipython-input-31-4bd4a7eeb594>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filter_two_dropped_loser = filter_two_dropped[filter_one_dropped["winner"] != "W"]


In [32]:
# Here, since we have already searched through the list of house winners for people by their
# first and last names, then we need to take a look at the people that were not coded as winners

loser_merge = second_end.merge(filter_two_dropped_loser, how="right", left_on=['fname','lname', 'district', 'state_po', 'cycle']\
                      , right_on = ['fname','lname','district', 'state_po', 'year'])

loser_merge_drop = loser_merge.drop_duplicates(subset=['fname','lname','district', 'state_po', 'year'], keep='first')
loser_merge_drop = loser_merge_drop.drop(['winner_y','name_y','cycle_y'], axis=1)
loser_merge_drop = loser_merge_drop.rename(columns={"name_x": "name", "cycle_x": "cycle", "winner_x": "winner"})

,district,state_po,cycle,fecyear,ICPSR,ICPSR2,name,lname,party,Incum.Chall,...,candidate,writein,mode,totalvotes,unofficial,version,fusion_ticket,candidatevotes,fixed_candidate,fname
1,01,AK,NaN,NaN,NaN,NaN,NaN,young,NaN,NaN,...,donald e. young,False,TOTAL,154618,False,20210604,False,114089,donald young,donald
7,15,CA,1980.0,1980.0,146131980,14613,"coelho, anthony lee",coelho,100,I,...,tony coelho,False,TOTAL,150491,False,20210604,False,108072,tony coelho,anthony
9,26,CA,NaN,NaN,NaN,NaN,NaN,rousselot,NaN,NaN,...,john h. rousselot,False,TOTAL,164514,False,20210604,False,116715,john rousselot,john
12,01,CO,1980.0,1980.0,140511980,14051,"schroeder, patricia",schroeder,100,I,...,pat schroeder,False,TOTAL,179622,False,20210604,False,107364,pat schroeder,patricia
17,11,FL,1980.0,1980.0,146481980,14648,"mica, daniel andrew",mica,100,I,...,dan mica,False,TOTAL,339233,False,20210604,False,201713,dan mica,daniel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2756,02,WV,NaN,NaN,NaN,NaN,NaN,mooney,NaN,NaN,...,alexander x. mooney,False,TOTAL,242014,False,20210604,False,140807,alexander mooney,alexander
2758,01,WI,NaN,NaN,NaN,NaN,NaN,ryan,NaN,NaN,...,paul d. ryan,False,TOTAL,354245,False,20210604,False,230072,paul ryan,paul
2762,05,WI,NaN,NaN,NaN,NaN,NaN,sensenbrenner,NaN,NaN,...,f. james sensenbrenner jr.,False,TOTAL,390844,False,20210604,False,260706,f. sensenbrenner,f
2763,06,WI,NaN,NaN,NaN,NaN,NaN,grothman,NaN,NaN,...,glenn grothman,False,TOTAL,357183,False,20210604,False,204147,glenn grothman,glenn
